In [60]:
import numpy as np
from tqdm import tqdm
from sklearn.linear_model import Perceptron
import copy
import math
import cvxpy as cp

In [75]:
class Gentile():
    def __init__(self, p, alpha, B, C):
        self.alpha = alpha
        self.B = B
        self.C = C
        self.p = p
        self.q = self.p/(self.p - 1)
        
    def solve(self, X, y):
        dim = X.shape[1]
        self.initialize(dim)
        for idx in tqdm(range(10000)):
            self.forward(X, y)
            accuracy_metric = np.sum(y * (X @ self.weights) > 0)/X.shape[0]
            if accuracy_metric == 1.0:
                break
        return self.weights, idx
    

    
    
    def initialize(self, dim):
        self.weights = np.zeros(dim)
        self.k = 1
        
    def forward(self, X, y):
        self.gamma = self.B * np.sqrt(self.p - 1) * 1/np.sqrt(self.k)
    
        
        for t in range(X.shape[0]):
            if y[t] * np.dot(self.weights,X[t]) <= (1 - self.alpha) * self.gamma:
                self.eta = self.C/np.sqrt(self.p - 1) * 1/np.sqrt(self.k)
                w_prime = self.finv(self.f(self.weights)+ self.eta * y[t] * X[t])
                q_norm = np.linalg.norm(w_prime, ord=self.q)
                self.weights = w_prime/max(1, q_norm)
                self.k += 1
            
                
    def f(self, w):
        numerator = np.sign(w) * np.power(abs(w), self.q - 1)
        denominator = np.power(np.linalg.norm(w, ord=self.q), self.q - 2)
        return numerator/denominator
    
    def finv(self, theta):
        numerator = np.sign(theta) * np.power(abs(theta), self.p - 1)
        denominator = np.power(np.linalg.norm(theta, ord=self.p), self.p - 2)
        return numerator/denominator
        
    

In [76]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference

class Pnorm_accelerated():
    def __init__(self, p):

        self.p = p
        self.q = p/(p-1)
        self.eta_w = 1/(2 * self.q - 2)
    
    def initialize(self, X, y):
        self.ws = np.array([])
        self.probs = np.array([])
        self.probs = np.append(self.probs, np.asarray([1] * X.shape[0])/X.shape[0])
        self.probs = self.probs.reshape(1, X.shape[0])
        
    
    def iterate(self, X, y):
        self.initialize(X, y)
        M = X * y[:, np.newaxis]
        percent_accuracy = 0
        for t in tqdm(range(1000), desc="Acc: {0:.3g}".format(percent_accuracy)):
            w = cp.Variable(len(X[0]))
            banana = -1 * np.sum(self.probs @ M, axis=0) @ w + -1 * self.probs[-1] @ M @ w + self.eta_w/2 * cp.square(cp.atoms.pnorm(w))
            problem = cp.Problem(cp.Minimize(banana) )
            try:
                problem.solve()
            except:
                problem.solve(solver="SCS")
            self.w = w.value
            
            if len(self.ws) == 0:
                self.ws = np.asarray(self.w).reshape(1, X.shape[1])
            else:
                self.ws = np.vstack([self.ws, self.w.reshape(1, X.shape[1])])
            
            prob_weightings = -1/2 * np.sum(M @ self.ws.T, axis=1) - 1
#             prob_weightings = prob_weightings/abs(np.sum(prob_weightings))
            self.prob = softmax(-M@np.sum(self.ws,axis=0)/2 - 1)
            self.probs = np.vstack([self.probs, self.prob.reshape(1, X.shape[0])])
            if np.isnan(self.probs).any():
                breakpoint()  
            percent_accuracy = np.sum(M @ np.mean(pna.ws, axis=0).T > 0)/M.shape[0]
            if((M @ np.mean(pna.ws, axis=0).T > 0).all()):
                break
                
        return np.mean(pna.ws, axis=0), t

In [86]:
def get_dataset_diff(dim):
    n = 10
    points = np.zeros((n,n))
    labels = np.zeros(n)
    for i in range(n):
        points[i,:i] = (-1)**(i+1)
        points[i,i] = (-1)**i
    #     points[i,:] = points[i]/np.linalg.norm(points[i])
        labels[i] = (-1)**i
    return points, labels
def get_dataset_easy(d):
    margin = 0.01
    n = 1000000
    w = np.random.rand(d)*2-1
    points = np.random.rand(n,d)-0.5
    points = points/np.linalg.norm(points, axis=1, keepdims=True)
    labels = np.sign(points@w+1e-32)[:,np.newaxis]
        
    # Preprocess for positive labels
    points = points*labels
    points = points[points@w>margin,:]
    labels = np.ones(points.shape[0])
    
    return points, labels

In [ ]:
X, y = get_dataset_easy(100)
Xb, yb = get_dataset_diff(100)
gentile = Gentile(3, .5, 1, 1)
pna = Pnorm_accelerated(2)

pna = Pnorm_accelerated(2)
# X, y= get_dataset(100)
M = X * y[:, np.newaxis]
final_w_pna, pna_num_iter = pna.iterate(X, y)
final_w_gentile, gentile_num_iter = gentile.solve(X, y)
breakpoint()
print('PNA: {}, Gentile: {}'.format(pna_num_iter, gentile_num_iter))


  0%|          | 1/10000 [01:00<169:22:29, 60.98s/it]


--Return--
None
> <ipython-input-89-9e57efb39389>(11)<module>()
      8 M = X * y[:, np.newaxis]
      9 final_w_pna, pna_num_iter = pna.iterate(X, y)
     10 final_w_gentile, gentile_num_iter = gentile.solve(X, y)
---> 11 breakpoint()
     12 print('PNA: {}, Gentile: {}'.format(pna_num_iter, gentile_num_iter))

ipdb> X
array([[ 0.16009014, -0.02293669, -0.08993194, ..., -0.05658431,
        -0.1056377 , -0.09923768],
       [-0.15333879, -0.09572267,  0.05292392, ...,  0.13795559,
         0.04654987,  0.11230934],
       [-0.11678756, -0.11863988, -0.15506677, ...,  0.03039773,
        -0.13264117, -0.06648192],
       ...,
       [-0.15203554,  0.09334374,  0.14742443, ..., -0.10381549,
        -0.02061074,  0.09731791],
       [ 0.11271763,  0.1615603 , -0.01386434, ...,  0.00692641,
         0.17482005,  0.17122786],
       [-0.05969629, -0.06495025, -0.0756351 , ...,  0.1454548 ,
         0.14650997, -0.00046862]])
ipdb> y
array([1., 1., 1., ..., 1., 1., 1.])
ipdb> M
array([[ 0.1

In [72]:
num_examples = 100
# dim = 5
# X = np.random.rand(num_examples, dim)
# y = np.random.choice([-1, 1], (num_examples))
X, y= get_dataset(num_examples)
final_weights = gentile.solve(X, y)
accuracy_metric = np.sum(y * (X @ final_weights) > 0)/X.shape[0]
print(y * X @ final_weights)
clf = Perceptron(tol=1e-3, random_state=0, fit_intercept=False)
clf.fit(X, y)
breakpoint()
accuracy_metric_scipy = y * np.squeeze(X @ clf.coef_.T)

100%|██████████| 10000/10000 [00:01<00:00, 5477.89it/s]
/Users/etashguha/anaconda3/envs/Research/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 100)